### Clustering Wrangle Exercises

#### Acquire

Zillow

For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline
pd.options.display.float_format = '{:20,.2f}'.format

import env

In [35]:
def get_db_url(database):
    url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/{database}'
    return url

def get_zillow():
    sql = '''
SELECT properties_2017.*, predictions_2017.logerror, 
predictions_2017.transactiondate, 
airconditioningtype.airconditioningdesc, architecturalstyletype.architecturalstyledesc,
buildingclasstype.buildingclassdesc, heatingorsystemtype.heatingorsystemdesc,
propertylandusetype.propertylandusedesc, storytype.storydesc, 
typeconstructiontype.typeconstructiondesc
FROM properties_2017
LEFT JOIN airconditioningtype USING(airconditioningtypeid)
LEFT JOIN architecturalstyletype USING(architecturalstyletypeid)
LEFT JOIN buildingclasstype USING(buildingclasstypeid)
LEFT JOIN heatingorsystemtype USING(heatingorsystemtypeid)
LEFT JOIN propertylandusetype USING(propertylandusetypeid)
LEFT JOIN storytype USING(storytypeid)
LEFT JOIN typeconstructiontype USING(typeconstructiontypeid)
LEFT JOIN predictions_2017 USING(parcelid)
WHERE predictions_2017.transactiondate LIKE '2017%%';
    '''
    url = get_db_url('zillow')
    zillow_df = pd.read_sql(sql, url, index_col = 'id')
    return zillow_df

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

 - Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
 - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
 - Only include properties that include a latitude and longitude value.

In [36]:
df = get_zillow()

2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77613 entries, 1727539 to 1187175
Data columns (total 67 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77613 non-null  int64  
 1   airconditioningtypeid         25007 non-null  float64
 2   architecturalstyletypeid      207 non-null    float64
 3   basementsqft                  50 non-null     float64
 4   bathroomcnt                   77579 non-null  float64
 5   bedroomcnt                    77579 non-null  float64
 6   buildingclasstypeid           15 non-null     float64
 7   buildingqualitytypeid         49809 non-null  float64
 8   calculatedbathnbr             76963 non-null  float64
 9   decktypeid                    614 non-null    float64
 10  finishedfloor1squarefeet      6037 non-null   float64
 11  calculatedfinishedsquarefeet  77378 non-null  float64
 12  finishedsquarefeet12          73923 non-null  float6

In [14]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
parcelid,"77,381.00","13,007,150.51","3,481,345.65","10,711,855.00","11,538,305.00","12,531,568.00","14,211,831.00","167,689,317.00"
airconditioningtypeid,"24,953.00",1.81,2.97,1.00,1.00,1.00,1.00,13.00
architecturalstyletypeid,206.00,7.39,2.73,2.00,7.00,7.00,7.00,21.00
basementsqft,50.00,679.72,689.70,38.00,273.00,515.00,796.50,"3,560.00"
bathroomcnt,"77,381.00",2.30,1.00,0.00,2.00,2.00,3.00,18.00
bedroomcnt,"77,381.00",3.05,1.14,0.00,2.00,3.00,4.00,16.00
buildingclasstypeid,15.00,3.93,0.26,3.00,4.00,4.00,4.00,4.00
buildingqualitytypeid,"49,672.00",6.53,1.72,1.00,6.00,6.00,8.00,12.00
calculatedbathnbr,"76,772.00",2.32,0.98,1.00,2.00,2.00,3.00,18.00
decktypeid,614.00,66.00,0.00,66.00,66.00,66.00,66.00,66.00


3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [19]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing

In [26]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
parcelid,0,0.00
airconditioningtypeid,52428,67.75
architecturalstyletypeid,77175,99.73
basementsqft,77331,99.94
bathroomcnt,0,0.00
...,...,...
buildingclassdesc,77366,99.98
heatingorsystemdesc,27941,36.11
propertylandusedesc,0,0.00
storydesc,77331,99.94


4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [22]:
def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})\
    .reset_index()\
    .groupby(['num_cols_missing', 'percent_cols_missing']).count()\
    .rename(index=str, columns={'customer_id': 'num_rows'}).reset_index()
    return rows_missing.T

In [27]:
nulls_by_row(df)

,num_cols_missing,percent_cols_missing,id
0,23,34.32835820895522,2
1,24,35.82089552238806,13
2,25,37.3134328358209,24
3,26,38.80597014925373,65
4,27,40.298507462686565,316
5,28,41.7910447761194,455
6,29,43.28358208955223,5270
7,30,44.776119402985074,3455
8,31,46.26865671641791,9891
9,32,47.76119402985074,12579


#### Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).